In [123]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [156]:
raw_data_path = '../data/raw/'

name = '17punto10'
reviews_raw = pd.read_csv(raw_data_path + 'collected_reviews_' + name + '.csv')
resumme_raw = pd.read_csv(raw_data_path + 'resumme_' + name + '.csv')
display(resumme_raw)
display(reviews_raw.sample(5))

,stars,reviews
0,5,571
1,4,191
2,3,35
3,2,13
4,1,13


,author,local_guide_info,rating,review,date_text,text_backup
15,Javier Fauro,Local Guide · 311 reseñas · 2.391 fotos,5 estrellas,"Platos muy elaborados, sabores espectaculares!...",Hace 5 meses,Javier Fauro\nLocal Guide · 311 reseñas · 2.39...
9,Daniel Sánchez-Rubio,Local Guide · 64 reseñas · 111 fotos,4 estrellas,Es un sitio donde la comida es sencillamente d...,Hace un año,Daniel Sánchez-Rubio\nLocal Guide · 64 reseñas...
7,Cristina Q,9 reseñas · 14 fotos,5 estrellas,Sitio de nivel. La comida relación calidad-pre...,Hace 8 meses,Cristina Q\n9 reseñas · 14 fotos\n\n\n\n\n...
27,Andrés Fdez,33 reseñas · 12 fotos,5 estrellas,"Ambiente muy bueno, sin ruidos y con un person...",Hace un año,Andrés Fdez\n33 reseñas · 12 fotos\n\n\n\n...
18,Gema Pérez,Local Guide · 54 reseñas · 192 fotos,5 estrellas,Atención y comida maravillosa.\nNo sabría eleg...,Hace un año,Gema Pérez\nLocal Guide · 54 reseñas · 192 fot...


In [154]:
raw_data_path = '../data/raw/'
reviews_raw = pd.read_csv(raw_data_path + 'collected_data.csv')
reviews_raw.drop(columns=['price', 'food', 'service', 'atmosphere', 'recommendations'], inplace=True)
reviews_raw.rename(columns = {'date':'date_text'}, inplace = True)
display(reviews_raw.sample(5))

,author,local_guide_info,rating,review,date_text,text_backup
123,ANI,Local Guide · 210 reseñas · 170 fotos,5 estrellas,Excelente lugar! Acompañando a la bebida te si...,Hace 2 años,ANI\nLocal Guide · 210 reseñas · 170 fotos\n\...
234,Manuel Alvarez Jara,Local Guide · 219 reseñas · 1.302 fotos,4 estrellas,Repetiré.,Hace 5 años,Manuel Alvarez Jara\nLocal Guide · 219 reseñas...
144,Carmen Garcia,Local Guide · 13 reseñas · 7 fotos,5 estrellas,Hemos venido por recomendación y ha superado n...,Hace 7 meses,Carmen Garcia\nLocal Guide · 13 reseñas · 7 fo...
225,P Gallardo,Local Guide · 161 reseñas · 55 fotos,4 estrellas,"Muy visitado por jóvenes estudiantes, podrás s...",Hace 7 años,P Gallardo\nLocal Guide · 161 reseñas · 55 fot...
118,Amanda Gonzalez,3 reseñas,1 estrella,La bebida y la comida espectacular. Pero los e...,Hace 6 meses,Amanda Gonzalez\n3 reseñas\n\n\n\n\n\n\n...


In [157]:
restaurant_search_words = {
    'service': r'Servicio\n([^\n]+)',
    'meal_type': r'Tipo de comida\n([^\n]+)',
    'price_per_person': r'Precio por persona\n([0-9€\- ]+)',
    'food_score': r'Comida: (\d+)',
    'service_score': r'Servicio: (\d+)',
    'atmosphere_score': r'Ambiente: (\d+)',
    'recommended': r'Platos recomendados\n([^\n]+)'
}

In [158]:
def extractRestaurantDetailsFromReview(sample, search_words=None, verbose=False):
    clean_text = re.sub(r'\\ue[0-9a-f]{3}', '', sample)
    clean_text = re.sub(r'\n+', '\n', clean_text)
    clean_text = clean_text.strip()

    # Store extracted values
    extracted_values = []

    # Loop through search words to extract values dynamically
    for key, regex in search_words.items():
        match = re.search(regex, clean_text)
        value = match.group(1) if match else ''
        extracted_values.append(value)

    return extracted_values

def applyExtractDetails(df, search_words=None):
    column_names = list(search_words.keys())
    df[column_names] = df['text_backup'].apply(lambda x: pd.Series(extractRestaurantDetailsFromReview(x, search_words=search_words)))
    return df

def extractReviewCount(text):
    if isinstance(text, str):  # Verify if its a string
        match = re.search(r'(\d+)\s+reseñas', text)
        if match:
            return int(match.group(1))
    return None

def extractStarRating(text):
    match = re.search(r'(\d+)\s+estrellas', text)
    if match:
        return int(match.group(1))
    return None

def extractRecommendations(recommendations):
    recommendations_list = recommendations.split(', ')
    if ' y ' in recommendations_list[-1]:
        last_dishes = recommendations_list[-1].rsplit(' y ', 1)
        recommendations_list = recommendations_list[:-1] + last_dishes
    return recommendations_list

def convertToDate(date_text):
    today = datetime.today()

    if 'semana' in date_text:
        # Extract number of weeks, default to 1 if no number is present
        weeks = pd.Series(date_text).str.extract(r'(\d+)')[0]
        weeks = int(weeks.iloc[0]) if pd.notna(weeks.iloc[0]) else 1
        monday_of_current_week = today - timedelta(days=today.weekday())  # Get Monday of the current week
        return monday_of_current_week.date() - timedelta(weeks=weeks)

    elif 'mes' in date_text:
        # Extract number of months, default to 1 if no number is present
        months = pd.Series(date_text).str.extract(r'(\d+)')[0]
        months = int(months.iloc[0]) if pd.notna(months.iloc[0]) else 1
        target_date = today - relativedelta(months=months)
        # Return the first day of the target month
        return target_date.replace(day=1).date()

    elif 'año' in date_text:
        # Extract number of years, default to 1 if no number is present
        years = pd.Series(date_text).str.extract(r'(\d+)')[0]
        years = int(years.iloc[0]) if pd.notna(years.iloc[0]) else 1
        target_date = today - relativedelta(years=years)
        # Return the first day of the target year
        return target_date.replace(month=1, day=1).date()

    return None  # Return None if no match is found


In [159]:
reviews = reviews_raw.copy()

reviews['local_guide_reviews'] = reviews['local_guide_info'].apply(extractReviewCount)
reviews['rating_score'] = reviews['rating'].apply(extractStarRating)
reviews = applyExtractDetails(reviews, search_words = restaurant_search_words)
reviews['recommendations_list'] = reviews['recommended'].apply(extractRecommendations)
reviews['date'] = reviews['date_text'].apply(convertToDate)

reviews['food_score'] = reviews['food_score'].apply(pd.to_numeric, errors='coerce')
reviews['service_score'] = reviews['service_score'].apply(pd.to_numeric, errors='coerce')
reviews['atmosphere_score'] = reviews['atmosphere_score'].apply(pd.to_numeric, errors='coerce')
reviews['avg_price_per_person'] = reviews['price_per_person'].str.extract(r'-(\d+)\s*€')
reviews['avg_price_per_person'] = pd.to_numeric(reviews['avg_price_per_person'], errors='coerce').astype('Int64')


reviews.drop(columns = ['text_backup', 'local_guide_info', 'rating', 'author', 'recommended', 'date_text'], inplace = True)
reviews.reset_index(inplace=True)
reviews.rename(columns={'index': 'review_id', 'price_per_person':'price_per_person_category'}, inplace=True)

In [160]:
# Fill NA values
reviews['local_guide_reviews'] = reviews['local_guide_reviews'].fillna(1)
reviews['rating_score'] = reviews['rating_score'].fillna(1)

In [161]:
display(reviews.head(30))

,review_id,review,local_guide_reviews,rating_score,service,meal_type,price_per_person_category,food_score,service_score,atmosphere_score,recommendations_list,date,avg_price_per_person
0,0,Espectacular!!! Probablemente uno de los resta...,35,5,,,30-40 €,5.0,5.0,5.0,[],2024-07-01,40
1,1,"La comida es excepcional! Las migas, las croqu...",28,5,,,,5.0,4.0,4.0,[],2024-07-01,<NA>
2,2,Cenamos en familia y fue una experiencia gastr...,3,5,,,20-30 €,5.0,5.0,5.0,"[Brandada de Bacalao, Cremoso de Chocolate Bla...",2024-08-01,30
3,3,Espacio confortable aún estando en la barra ja...,16,5,,,20-30 €,5.0,5.0,5.0,[Tarta de Manzana Al Revés Con Helado de Canela],2024-05-01,30
4,4,SIEMPRE DE 10!\nHemos venido varias veces y ta...,54,5,,,,5.0,5.0,5.0,[Tarta de Manzana Al Revés Con Helado de Canel...,2024-08-26,<NA>
5,5,Fuimos recomendados por un amigo y fue todo un...,158,5,,,30-40 €,5.0,5.0,5.0,[],2024-03-01,40
6,6,Un sitio muy interesante para visitar. El serv...,103,5,,,20-30 €,5.0,5.0,5.0,"[Cremoso de Chocolate Blanco, Burrata Con Toma...",2024-04-01,30
7,7,Sitio de nivel. La comida relación calidad-pre...,9,5,,,,5.0,5.0,5.0,[Panceta Asada Mojo Rojo Y Chimichurri Casero ...,2024-01-01,<NA>
8,8,"Fuimos de celebración,el sitio es pequeño, de ...",40,5,,,30-40 €,5.0,5.0,5.0,[],2024-03-01,40
9,9,Es un sitio donde la comida es sencillamente d...,64,4,,,,5.0,5.0,5.0,[],2023-01-01,<NA>
